# Key word extraction algorithm for claims

In [1]:
import os, json
from helper_function import count_valid_posts, get_claims, get_premises
import pandas as pd
import numpy as np
from nltk.corpus import stopwords

In [2]:
with open('../data/v3_70_filled/Should-I-invest-in-Bitcoin_with_labels_v3_70_filled.json', 'r') as f:
    file_content = json.load(f)
print('Get valid posts: ', count_valid_posts(file_content))
claims = get_claims(file_content)
premises = get_premises(file_content)

Get valid posts:  (41, 70)


## Keyword extraction algorithms

In [19]:
## TD-IDF
from sklearn.feature_extraction.text import TfidfVectorizer

tf_idf_vectorizer = TfidfVectorizer(stop_words=stopwords.words('english'))
vectors = tf_idf_vectorizer.fit_transform(premises)
feature_names = tf_idf_vectorizer.get_feature_names()
dense = vectors.todense()


In [20]:
dense

matrix([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]])

In [21]:
np.argsort(dense, axis=1)

matrix([[  0, 501, 502, ..., 238, 369, 140],
        [  0, 499, 500, ..., 531, 695, 702],
        [  0, 499, 500, ..., 268, 556, 494],
        ...,
        [  0, 500, 501, ..., 760, 559, 603],
        [  0, 502, 503, ..., 250, 760, 443],
        [  0, 501, 502, ..., 105, 730, 249]])

In [22]:
x = np.asarray(np.argsort(dense, axis=1)[:, :-4:-1])


In [23]:
m = []
for i in x:
    feature = [feature_names[y] for y in i]
    m.append(feature)
m

[['closed', 'issue', 'everything'],
 ['understand', 'trouble', 'quickly'],
 ['phrase', 'research', 'focused'],
 ['short', 'term', 'longer'],
 ['sense', 'invest', 'jumped'],
 ['market', 'earn', 'studying'],
 ['gold', 'digital', 'like'],
 ['gold', 'long', 'less'],
 ['verified', 'addition', 'predictable'],
 ['created', 'bitcoins', 'see'],
 ['anywhere', 'world', 'account'],
 ['created', 'bitcoins', 'features'],
 ['failed', 'escape', 'border'],
 ['global', 'information', 'access'],
 ['market', 'price', 'investment'],
 ['hacking', 'exchanges', 'threat'],
 ['little', 'operates', 'stand'],
 ['besides', 'legitimate', 'accepted'],
 ['bankrupt', 'recover', 'corrupts'],
 ['useless', 'pretty', 'gold'],
 ['brand', 'gold', 'world'],
 ['world', 'one', 'fast'],
 ['banks', 'clients', 'hnw'],
 ['gold', 'cared', 'forget'],
 ['previous', 'upside', 'believe'],
 ['futures', 'inception', 'cap'],
 ['value', 'answer', 'considering'],
 ['dollars', 'ecosystem', 'increase'],
 ['addresses', 'active', 'institutional